# Stock Anlysis

In [10]:
import pandas as pd
import yfinance as yf
from datetime import date, timedelta

# importing list of stock tickers
NASDAQ_tickers_list = pd.read_csv('nasdaq_tickers.csv')

# getting dates to observe (past 30 days)
todays_date = date.today()
from_date = date.today() - timedelta(days=30)

# creating dictionary to store dataframes
stock_data = {}

for stock in NASDAQ_tickers_list['Symbol']:
    try:
        data = stock_data[stock] = yf.download(stock, from_date, todays_date, progress=False)
        if not data.empty:
            stock_data[stock] = {
                'company_name': yf.Ticker(stock).info['longName'],
                'data': data
            }
    except Exception as e:
        print(f"Error fetching data for {stock}: {e}")



1 Failed download:
['AACIW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['AAGRW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['ABLLW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['ABVEW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['ACONW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['ADNWW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['ADSEW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['ADVWW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['AENTW']: YFInvalidPeriodError("%ti

Error fetching data for DHCNI: 'longName'



1 Failed download:
['DPCSW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['DRTSW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['DSYWW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['DTSTW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['DYCQR']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['ECDAW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['ECXWW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['EDBLW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['EMLDW']: YFInvalidPeriodError("%ti

Error fetching data for GECCM: 'longName'



1 Failed download:
['GECCZ']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['GFAIW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['GGROW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['GHIXW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['GIPRW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['GLLIR']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['GLSTR']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['GOEVW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['GOVXW']: YFInvalidPeriodError("%ti

...


1 Failed download:
['TRONW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['TVGNW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['UHGWW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['UKOMW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['USGOW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['VGASW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['VMCAW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['VSACW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['VSEEW']: YFInvalidPeriodError("%ti

Error fetching data for XYLO: 'longName'



1 Failed download:
['YOTAW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['ZAPPW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['ZCARW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['ZEOWW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")


Error fetching data for ZIONL: 'longName'



1 Failed download:
['ZOOZW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['ZPTAW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['ZURAW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
